<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_Scilab_to_Python_versao_05_fev_18_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
#Enxame retorna a populacao aleatória com todos valores entre MIN e MAX
def Enxame(numParametros,numParticulas,deltaMax,deltaMin):
  x=np.zeros((numParticulas, numParametros))
  for j in range(numParametros):
    for i in range(numParticulas):
      r = np.random.random()
      x[i,j]=deltaMin[j]+(deltaMax[j]-deltaMin[j])*r
  return x

In [0]:
#FOBJ calcula o valor da função a ser minimizada para cada partícula
def FOBJ(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF):
  rows,cols=x.shape
  fobjr=np.zeros((rows))
  for i in range(rows):
    # fobjr[i]=FUN(x[i,:],mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
    fobjr[i]=FUN(x[i,:],mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
  return fobjr

In [0]:
#FUN calcula o TAC para um determinado arranjo
#function fun=FUN(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
def FUN(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF):
  funr=0
  #latenteH2O=2256 #[kJ/kg] (vap sat, 2 bar)
  #cpH2O=4.1868 #[kJ/(kg.K)]
  # print(mCp)
  # rows,cols=mCp.shape 
  cols=len(mCp)
  matrizDeltaT=np.zeros((cols,cols))
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
  contador=0
  matrizBinaria=np.zeros((cols,cols))
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizBinaria[i,j]=round(x[contador+6])
        contador=contador+1
    
  areaTrocador=0;tempSaidaQ=0;tempSaidaF=0;deltaTMediaLog=0;
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizBinaria[i,j]==1):
          if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
            #i: CQ, j: CF
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                tempAtual[i]>tempAtual[j] and 
                tempSaidaQ>tempAtual[j] and 
                tempAtual[i]>tempSaidaF
              ):
              #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
              A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
              B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
              deltaTMediaLog=A/B
              areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF
          else:#a corrente i ganha calor da j
            #i: CF, j: CQ
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                  
                 tempAtual[i]< tempAtual[j] and
                 tempSaidaF<tempAtual[j] and
                 tempAtual[i]<tempSaidaQ

              ):#TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
              A=((tempSaidaQ-tempSaidaF)-(tempAtual[j]-tempAtual[i]))
              B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[j]-tempAtual[i]))
              deltaTMediaLog=A/B
              areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
              tempAtual[i]=tempSaidaF
              tempAtual[j]=tempSaidaQ
                            
                      
   
  cargaUQ=0; cargaUF=0;
  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      #UQ -> vapor saturado -> entra e sai a 100ºC
      A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
      B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
      deltaTMediaLog=A/B
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    else:#usa UF      
      cargaUF=cargaUF+mCp[i]*(-deltaT)
      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      A=((tempAlvo[i]-293.15)-(tempAtual[i]-278.15))
      B=np.log((tempAlvo[i]-293.15)/(tempAtual[i]-278.15))
      deltaTMediaLog=A/B
      areaTrocador=areaTrocador+abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
 
  costTrocador=300*(areaTrocador)**0.5
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF
  funr=costUQ+costUF+costTrocador

  return funr


In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  # contador=-1
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        # contador=contador+1
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
    
    '''
    nao tem esta linha (contador=0) abaixo 
    no original mas se tirar ela gera out of bounds
    '''
    contador=0 
    matrizBinaria=np.zeros((cols,cols))
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          # matrizBinaria[i,j]=round(x[contador-1])
          '''
          parece q faz mais sentido assim (forma abaixo)
          '''
          matrizBinaria[i,j]=round(x[contador+nref])
          # print('matriz binaria',matrizBinaria[i,j])
          contador=contador+1
     
    tempSaidaQ=0;tempSaidaF=0;cargaTermica=0
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j 
              #i: CQ, j: CF
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
                ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
                tempAtual[i]=tempSaidaQ
                tempAtual[j]=tempSaidaF
              else:
                matrizDeltaT[i,j]=0
            else:#a corrente i ganha calor da j
              # i: CF, j:CQ
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]<tempAtual[j] and
                  tempSaidaF<tempAtual[j] and
                  tempAtual[i]<tempSaidaQ
                 ):
                #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
                tempAtual[i]=tempSaidaF
                tempAtual[j]=tempSaidaQ
              else:
                matrizDeltaT[i,j]=0
          else:
            matrizDeltaT[i,j]=0
  
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:

#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  matrizGrafico=np.zeros((cols,cols))
  contador=1
  matrizGrafico[:,0]=tempAtual
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
          #i: CQ, j:CF
          cargaTermica=mCp[i]*matrizDeltaT[i,j]
          tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
          if(
              tempAtual[i]>tempAtual[j] and
              tempSaidaQ>tempAtual[j] and
              tempAtual[i]>tempSaidaF
              
            ):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
            matrizGrafico[:,contador]=tempAtual
            contador=contador+1
            
            #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
            contador=contador+1
            # troquei o elif por else (fica melhor no contexto?)
            # elif (matrizDeltaT[i,j]>0) : #a corrente i ganha calor da j
          else : #a corrente i ganha calor da j
            # situa
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j] 
            if (
                
                tempAtual[i]<tempAtual[j] and 
                tempSaidaF<tempAtual[j] and 
                tempAtual[i]<tempSaidaQ

               ):
              #  i: CF, j: CQ
              
              tempAtual[i]=tempSaidaF
              tempAtual[j]=tempSaidaQ
              matrizGrafico[:,contador]=tempAtual
              contador=contador+1          

  return matrizGrafico

In [0]:
#PART atualiza PBEST (melhor de cada particula)
#function PBEST=PART(x,xnew,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
def PART(x,xnew,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF):
  #rows,cols=x.shape()
  rows,cols=x.shape
  YCAL=FOBJ(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
  NEW=FOBJ(xnew,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF) #xnew vem da rotina velocidade
  for i in range(rows):
    if(YCAL[i]>NEW[i]):
      x[i,:]=xnew[i,:]
  PBEST=x
  return PBEST

In [0]:
#RESULT calcula o TAC, area e cargas de UQ e UF para um determinado arranjo
#function [TAC,areaTrocador,cargaUQ,cargaUF]=RESULT(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
def RESULT(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF):
  # latenteH2O=2256 #[kJ/kg] (vap sat, 2 bar)
  # cpH2O=4.1868 #[kJ/(kg.K)]
  print(x)
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)
  matrizDeltaT=np.zeros((cols,cols))
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1

  contador=0  
  matrizBinaria=np.zeros((cols,cols))
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1

  TAC=0;areaTrocador=0;tempSaidaQ=0;tempSaidaF=0;deltaTMediaLog=0;
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizBinaria[i,j]==1):#a corrente i perde calor para a j
          #i: CQ, j: CF
          if(matrizDeltaT[i,j]<0):
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                
               tempAtual[i]>tempAtual[j] and
               tempSaidaQ>tempAtual[j] and
               tempAtual[i]>tempSaidaF

              ):
              A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
              B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
              deltaTMediaLog=A/B
              areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF
        else:#a corrente i ganha calor da j
          #i: CQ, j: CF
          cargaTermica=mCp[i]*matrizDeltaT[i,j]
          tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
          if(
              tempAtual[i]<tempAtual[j] and 
              tempSaidaF<tempAtual[j] and
              tempAtual[i]<tempSaidaQ
            ):
            #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
            A=((tempSaidaQ-tempSaidaF)-(tempAtual[j]-tempAtual[i]))
            B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[j]-tempAtual[i]))
            deltaTMediaLog=A/B
            areaTrocador=areaTrocador+abs(cargaTermica/(U*deltaTMediaLog))
            tempAtual[i]=tempSaidaF
            tempAtual[j]=tempSaidaQ

  cargaUQ=0; cargaUF=0;
  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      #UQ -> vapor saturado -> entra e sai a 100ºC
      A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
      B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
      deltaTMediaLog=A/B
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    else:#usa UF
      cargaUF=cargaUF+mCp[i]*(-deltaT)
      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      A=((tempAlvo[i]-293.15)-(tempAtual[i]-278.15))
      B=np.log((tempAlvo[i]-293.15)/(tempAtual[i]-278.15))
      deltaTMediaLog=A/B
      areaTrocador=areaTrocador+abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
 
  costTrocador=300*(areaTrocador)**0.5
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF
  TAC=costUQ+costUF+costTrocador
  
  return TAC,areaTrocador,cargaUQ,cargaUF

In [0]:
'''
VE rotina velocidade utilizada na correcao das particulas
'''
#def VE(X,BEST,PBEST,W,C1,C2,MAX,MIN):
def VE(X,BEST,PBEST,W,C1,C2,MAX,MIN):
#def VE(X,VEL,BEST,PBEST,W,C1,C2,MAX,MIN):
    rows = X.shape[0]
    cols = X.shape[1]
    VEL=np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            R1=np.random.random()
            R2=np.random.random()
            VELOCIDADE=W*VEL[i,j]+C1*R1*(PBEST[i,j]-X[i,j])+C2*R2*(BEST[j]-X[i,j])
            if(VELOCIDADE> MAX[j]):
                VELOCIDADE=0.2*MAX[j]
            if(VELOCIDADE<MIN[j]):
                VELOCIDADE=0.2*MIN[j]
            VEL[i,j]=np.copy(VELOCIDADE)
            X[i,j]=X[i,j]+VEL[i,j]
    return X, VEL
    #return X

In [0]:
# function [BEST,GBEST,Vetor]=PSO(W,C1,C2,NPAR,ITE,PAR,MAX,MIN,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
def PSO(W,C1,C2,NPAR,ITE,PAR,MAX,MIN,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF):
  Vetor=np.zeros(ITE)
  x=Enxame(PAR,NPAR,MAX,MIN) #gera população inicial aleatória
  #print(x)
  PBEST=x #inicialmente as melhores partículas são aleatórias
  YCAL=FOBJ(x,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF) #estima a função nos valores aleatórios
  #print(YCAL)
  best=np.argmin(YCAL)
  GBEST=PBEST[best,:] #melhor global na 1a interação
  BEST=YCAL[best]
  VEL=Enxame(PAR,NPAR,MAX,MIN) #velocidades aleatórias na 1a iteração

  for i in range(ITE):
    xnew,VEL=VE(x,GBEST,PBEST,W,C1,C2,MAX,MIN)#nova atualização de x utilizando as velocidades   
    PBEST=PART(PBEST,xnew,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF) #atualização de PBEST (sempre atualiza com a nova busca)
    YCAL=FOBJ(PBEST,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)
    inew=np.argmin(YCAL)
    NEW=YCAL[inew]
    if(NEW<BEST):
      GBEST=xnew[new,:]
      BEST=NEW

    Vetor[i]=BEST;
    x=xnew

  return BEST,GBEST,Vetor

In [0]:

#********************PROGRAMA PRINCIPAL*************************
W=0.9
C1=2
C2=2
NPAR=80 #Número de partículas
ITE=100 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

#6 delta máximo/mínimo de cada parâmetro [ºC] e 6 coeficientes para matriz binária
MAX=np.array([60,60,60,115,115,0,1,1,1,1,1,1])
MIN=np.array([0,0,0,0,0,-120,0,0,0,0,0,0])

mCp=np.array([4,2,1.5,3]) #[kW/K]
tempAtual=np.array([453,393,523,533]) #[K]
tempAlvo=np.array([513,508,403,433]) #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

#Para garantir convergência
for i in range(6):
  MAX[i]=3*MAX[i]
  MIN[i]=-1*MAX[i]


BEST,GBEST, Vetor=PSO(W,C1,C2,NPAR,ITE,PAR,MAX,MIN,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)

Iter=range(ITE)


matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n',ITE,NPAR);print(GBEST)
print('Valor da função f(gbest)=%f \n',BEST)

print('Matriz de Trocas Térmicas \n')
print(matrizTrocaBest)
print('Matriz Binária \n')
print(matrizBinaria)

rows,cols=matrizGrafico.shape

indice=range(cols)
'''
for i in range(rows):
  plot(indice,matrizGrafico[i,:])
'''

# TAC=0;areaTrocador=0;cargaUQ=0;cargaUF=0;
TAC,areaTrocador,cargaUQ,cargaUF=RESULT(GBEST,mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF)

print('TESTE ADICIONAL')
print('Custo Total Anual= ',TAC)
print('Area de Trocador= ',areaTrocador)
print('Carga de UQ= ',cargaUQ)
print('Carga de UF= ',cargaUF)

#plot(Iter,Vetor,'black')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


Melhor solução para %d iterações com %d partículas 
 100 80
[-6.49906984e+01  1.67725275e+02  2.64488947e+01 -6.71998931e+01
  9.45779968e+01  0.00000000e+00  5.00854573e-01  6.34131284e-01
  6.93385850e-01  9.65027076e-01  4.75206529e-02  8.17358416e-01]
Valor da função f(gbest)=%f 
 nan
Matriz de Trocas Térmicas 

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Matriz Binária 

[[0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
[-6.49906984e+01  1.67725275e+02  2.64488947e+01 -6.71998931e+01
  9.45779968e+01  0.00000000e+00  5.00854573e-01  6.34131284e-01
  6.93385850e-01  9.65027076e-01  4.75206529e-02  8.17358416e-01]
TESTE ADICIONAL
Custo Total Anual=  30442.357198766294
Area de Trocador=  17.141178722302
Carga de UQ=  242.0
Carga de UF=  211.5


In [0]:
print(MAX)
print(MIN)
x=Enxame(PAR,NPAR,MAX,MIN)
print(matrizGrafico)

[180 180 180 345 345   0   1   1   1   1   1   1]
[-180 -180 -180 -345 -345    0    0    0    0    0    0    0]
[[471.   0.   0.   0.]
 [471.   0.   0.   0.]
 [470.   0.   0.   0.]
 [470.   0.   0.   0.]]


In [0]:
round(0.51)

1